In [91]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [92]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [93]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


In [94]:
data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

In [95]:
inputs = keras.Input(shape=(100, 100, 3))

x = data_augmentation(inputs)
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_129 (Conv2D)         (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_68 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_130 (Conv2D)         (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_69 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_131 (Conv2D)         (None, 21, 21, 128)       738

In [96]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_4.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

In [97]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 07:35:58.468521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


104/105 [============================>.] - ETA: 0s - loss: 2.5783 - accuracy: 0.2127

2022-08-01 07:36:04.324025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 7s 55ms/step - loss: 2.5696 - accuracy: 0.2125 - val_loss: 1.5605 - val_accuracy: 0.3500
Epoch 2/40
105/105 [==============================] - 5s 48ms/step - loss: 1.7578 - accuracy: 0.3446 - val_loss: 1.3096 - val_accuracy: 0.4722
Epoch 3/40
105/105 [==============================] - 5s 45ms/step - loss: 1.2986 - accuracy: 0.5366 - val_loss: 2.6733 - val_accuracy: 0.2972
Epoch 4/40
105/105 [==============================] - 5s 46ms/step - loss: 1.0280 - accuracy: 0.6354 - val_loss: 0.6464 - val_accuracy: 0.7931
Epoch 5/40
105/105 [==============================] - 5s 45ms/step - loss: 0.7389 - accuracy: 0.7405 - val_loss: 0.5478 - val_accuracy: 0.7778
Epoch 6/40
105/105 [==============================] - 5s 46ms/step - loss: 0.6182 - accuracy: 0.7940 - val_loss: 0.4949 - val_accuracy: 0.8194
Epoch 7/40
105/105 [==============================] - 5s 44ms/step - loss: 0.4578 - accuracy: 0.8357 - val_loss: 0.6731 - val_accuracy: 0.7792
Epoch 8/40

In [102]:
model.evaluate(test)

23/23 [==============================] - 0s 17ms/step - loss: 0.2292 - accuracy: 0.9222


[0.22924888134002686, 0.9222222566604614]

In [103]:
# keras.models.save_model(model, "./models/zenodo_3.h5")

load_model = keras.models.load_model("models/zenodo_4.h5")
load_model.evaluate(test)

 1/23 [>.............................] - ETA: 4s - loss: 0.1912 - accuracy: 0.9062

2022-08-01 08:02:13.457122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 18ms/step - loss: 0.2069 - accuracy: 0.9264


[0.2069445103406906, 0.9263889193534851]